In [1]:
import os

In [2]:
%pwd

'/Users/mayankchugh/gitRepos/mayankchugh.learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image/research'

In [3]:
%pwd

'/Users/mayankchugh/gitRepos/mayankchugh.learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image/research'

In [ ]:
os.chdir('d:\\git\\Chest-Disease-Classification-from-Chest-CT-Scan-Image\\')

In [4]:
%pwd

'/Users/mayankchugh/gitRepos/mayankchugh.learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image/research'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mayankchugh-learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mayankchugh-learning"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6e159232d9211a02a015f21fa7645986dfa6caa1"

## Entity Code

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## ConfigurationManager file code

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/mayankchugh-learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


